# Apache Spark

![Logo](../images/apache_spark_logo.png)

- [Apache Spark](https://spark.apache.org) was first released in 2014. 
- It was originally developed by [Matei Zaharia](http://people.csail.mit.edu/matei) as a class project, and later a PhD dissertation, at University of California, Berkeley.
- Spark is written in [Scala](https://www.scala-lang.org), but has a Python interface.

- Apache Spark is a fast and general-purpose cluster computing system. 
- It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs.
- Spark can manage "big data" collections with a small set of high-level primitives like `map`, `filter`, `groupby`, and `join`.  With these common patterns we can often handle computations that are more complex than map, but are still structured.
- It also supports a rich set of higher-level tools including [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html) for SQL and structured data processing, [MLlib](https://spark.apache.org/docs/latest/ml-guide.html) for machine learning, [GraphX](https://spark.apache.org/docs/latest/graphx-programming-guide.html) for graph processing, and Spark Streaming.

# Resilient distributed datasets

- The fundamental abstraction of Apache Spark is a read-only, parallel, distributed, fault-tolerent collection called a resilient distributed datasets (RDD).
- RDDs behave a bit like Python collections (e.g. lists).
- When working with Apache Spark we iteratively apply functions to every item of these collections in parallel to produce *new* RDDs.
- The data is distributed across nodes in a cluster of computers.
- Functions implemented in Spark can work in parallel across elements of the collection.
- The  Spark framework allocates data and processing to different nodes, without any intervention from the programmer.
- RDDs automatically rebuilt on machine failure.


# Lifecycle of a Spark Program
1. Create some input RDDs from external data or parallelize a collection in your driver program.
2. Lazily transform them to define new RDDs using transformations like `filter()` or `map()`
3. Ask Spark to cache() any intermediate RDDs that will need to be reused.
4. Launch actions such as count() and collect() to kick off a parallel computation, which is then optimized and executed by Spark.

# Operations on Distributed Data
- Two types of operations: **transformations** and **actions**
- Transformations are *lazy* (not computed immediately) 
- Transformations are executed when an action is run


# [Transformations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations) (lazy)
```spark
map() flatMap()
filter() 
mapPartitions() mapPartitionsWithIndex() 
sample()
union() intersection() distinct()
groupBy() groupByKey()
reduceBy() reduceByKey()
sortBy() sortByKey()
join()
cogroup()
cartesian()
pipe()
coalesce()
repartition()
partitionBy()
...
```

# [Actions](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions)

```
reduce()
collect()
count()
first()
take()
takeSample()
saveToCassandra()
takeOrdered()
saveAsTextFile()
saveAsSequenceFile()
saveAsObjectFile()
countByKey()
foreach()
```

# PySpark


PySpark uses Py4J that enables Python programs to dynamically access Java objects.

![PySpark Internals](http://i.imgur.com/YlI8AqEl.png)

## The `SparkContext` class

- When working with Apache Spark we invoke methods on an object which is an instance of the `pyspark.SparkContext` context.

- Typically, an instance of this object will be created automatically for you and assigned to the variable `sc`.

- The `parallelize` method in `SparkContext` can be used to turn any ordinary Python collection into an RDD;
    - normally we would create an RDD from a large file or an HBase table. 

## First example

PySpark isn't on sys.path by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding pyspark to sys.path at runtime. [findspark](https://github.com/minrk/findspark) does the latter.

In [ ]:
import findspark, pyspark

findspark.init()

sc = pyspark.SparkContext(master="local[2]", appName="FirstExample")

In [ ]:
sc.stop()

In [ ]:
# If tou get an error run this cell with the command below commented out
# and fix the path to spark and/or python in the cell above
# sc.stop()

We have a spark context sc to use with a tiny local spark cluster with 2 nodes (will work just fine on a multicore machine).

In [ ]:
print(sc) # it is like a Pool Processor executor

# Create your first RDD

In [ ]:
rdd = sc.parallelize(list(range(8))) # create collection

In [ ]:
rdd

In [ ]:
rdd.collect()  # Gather results back to local process

In [ ]:
square = rdd.map(lambda x: x ** 2) # Square each element

In [ ]:
square.collect() # Square each element and collect results

# Map-Reduce operation 

In [ ]:
from operator import add
rdd.map(lambda x: x ** 2).reduce(add) # reduce is an action!

# filter

In [ ]:
# Select only the even elements
rdd.filter(lambda x: x % 2 == 0).collect()

In [ ]:
# Cartesian product of each pair of elements in two sequences 
# (or the same sequence in this case)
rdd.cartesian(rdd).collect()

In [ ]:
# Chain operations to construct more complex computations
(rdd.map(lambda x: x ** 2)
    .cartesian(rdd)
    .filter(lambda tup: tup[0] % 2 == 0)
    .collect())

In [ ]:
# Stop the local spark cluster
sc.stop()

### Exercise 9.1 Word-count in Apache Spark

- Write the sample text file

In [ ]:
from lorem import text
with open('sample.txt','w') as f:
    f.write(text())


- Create the rdd with `SparkContext.textFile method`
- lower, remove dots and split using `rdd.flatMap`
- use `rdd.map` to create the list of key/value pair (word, 1)
- `rdd.reduceByKey` to get all occurences
- `rdd.takeOrdered`to get sorted frequencies of words

All documentation is available [here](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html?highlight=textfile#pyspark.SparkContext) for textFile and [here](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html?highlight=textfile#pyspark.RDD) for RDD. 

# Additional RDD transformations
Apache Spark offers many more methods for operating on collections of tuples over and above the standard Map-Reduce framework:
- Sorting: sortByKey, sortBy, takeOrdered
- Mapping: flatMap
- Filtering: filter
- Counting: count
- Set-theoretic: intersection, union
Many others: see the Transformations section of the [programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

## $\pi$ computation example

- We can estimate an approximate value for $\pi$ using the following Monte-Carlo method:


1.    Inscribe a circle in a square
2.    Randomly generate points in the square
3.    Determine the number of points in the square that are also in the circle
4.    Let $r$ be the number of points in the circle divided by the number of points in the square, then $\pi \approx 4 r$.
    
- Note that the more points generated, the better the approximation

See [this tutorial](https://computing.llnl.gov/tutorials/parallel_comp/#ExamplesPI).

In [ ]:
import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PythonPi").getOrCreate()

partitions = 4
n = 1000000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = spark.sparkContext.parallelize(range(1, n+1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))

spark.stop()

### Exercise 9.2

Using the same method than the PI computation example, compute the integral
$$
I = \int_0^1 \exp(-x^2) dx
$$
You can check your result with numpy

In [ ]:
# numpy evaluates solution using numeric computation. 
# It uses discrete values of the function
import numpy as np
x = np.linspace(0,1,1000)
np.trapz(np.exp(-x*x),x)

In [ ]:
# numpy and scipy evaluates solution using numeric computation. It uses discrete values
# of the function
import numpy as np
from scipy.integrate import quad
quad(lambda x: np.exp(-x*x), 0, 1)
# note: the solution returned is complex 

### Correlation between daily stock

- Data preparation

In [ ]:
from glob import glob
import os
import pandas as pd
import ujson as json # or json

def convert_to_json(d):
    """ Convert all csv files of directory d into json format """
    filenames = sorted(glob(os.path.join(d, '*')))[-365:]
    outfn = d.replace('minute', 'json') + '.json'
    if os.path.exists(outfn):
        return
    with open(outfn, 'w') as f:
        for fn in filenames:
            df = pd.read_csv(fn)
            for rec in df.to_dict(orient='records'):
                json.dump(rec, f)
                f.write('\n')
    print("Finished JSON: %s" % d.split(os.path.sep)[-1])

here = os.getcwd()
js = os.path.join(here, 'data', 'json')
if not os.path.exists(js):
    os.mkdir(js)

directories = sorted(glob(os.path.join(here, 'data', 'minute', '*')))
for d in directories:
    convert_to_json(d)

filenames = sorted(glob(os.path.join('data', 'json', '*.json')))

for fn in filenames:
    with open(fn) as f:
        data = [json.loads(line) for line in f]
        
    df = pd.DataFrame(data)
    
    out_filename = fn[:-5] + '.h5'
    df.to_hdf(out_filename, '/data')
    print("Finished : %s" % out_filename.split(os.path.sep)[-1])

filenames = sorted(glob(os.path.join('data', 'json', '*.h5')))  # ../data/json/*.json
filenames

### Sequential code

In [3]:
%%time

series = []
for fn in filenames:   # Simple map over filenames
    series.append(pd.read_hdf(fn)['close'])

results = []

for a in series:    # Doubly nested loop over the same collection
    for b in series:  
        if not (a == b).all():     # Filter out comparisons of the same series 
            results.append(a.corr(b))  # Apply function

result = max(results)

NameError: name 'filenames' is not defined

### Exercise 9.3

Parallelize the code above with Apache Spark.

Computation time is slower because there is a lot of setup, workers creation, there is a lot of communications the correlation function is too small

### Exercise 9.4 Fasta file example

We will use this RDD to calculate the frequencies of sequences of five bases, and then sort the sequences into descending order ranked by their frequency.
First we will define some functions to split the bases into sequences of a certain size:

- Create a rdd from fasta file nucleotide-sample.txt in data directory
- Use GroupBy to process every sequence present in the file.
- Transform the original text RDD into an RDD containing key-value pairs where the key is the sequence and the value is 1, as per the word-count example.
- Notice that if we simply `map` each line of text, we will obtain multi-dimensional data. 
- Use `flatMap` to flatten this data in order to turn it into a list of base-sequences. 
- Compute frequencies by using RDD methods `map`, `flatMap`, `reduceByKey`, `sortByKey` and `take`.
- Rank each sequence according to its count key, (first element) of each tuple should be the count. Reverse the tuples.
- Now we can sort the RDD in descending order of key: